## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-09-03-43-00 +0000,wsj,Singapore Dollar Strengthens Amid Risk-On Sent...,https://www.wsj.com/finance/currencies/yen-mos...
1,2026-02-09-03-40-16 +0000,nypost,Hundreds in Brooklyn without power as temperat...,https://nypost.com/2026/02/08/us-news/hundreds...
2,2026-02-09-03-37-09 +0000,nyt,"The 2026 Super Bowl Ads, Ranked",https://www.nytimes.com/2026/02/06/arts/televi...
3,2026-02-09-03-36-01 +0000,startribune,Seahawks ride their ‘Dark Side’ defense to a S...,https://www.startribune.com/drake-maye-and-the...
4,2026-02-09-03-35-05 +0000,nypost,‘Looksmaxxing’ influencer Clavicular arrested ...,https://nypost.com/2026/02/08/us-news/looksmax...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2444/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
65,trump,17
125,election,16
18,super,16
19,bowl,16
42,bad,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
14,2026-02-09-02-55-54 +0000,nypost,Trump slams Bad Bunny’s Super Bowl halftime sh...,https://nypost.com/2026/02/08/us-news/trump-sl...,101
10,2026-02-09-03-08-19 +0000,nyt,"Lady Gaga, Ricky Martin and Cardi B Join Bad B...",https://www.nytimes.com/2026/02/08/us/bad-bunn...,88
6,2026-02-09-03-23-45 +0000,nyt,Bad Bunny Delivers a Love Letter to Puerto Ric...,https://www.nytimes.com/2026/02/08/arts/music/...,83
5,2026-02-09-03-30-40 +0000,bbc,Bad Bunny piles on Puerto Rican patriotism at ...,https://www.bbc.com/news/articles/c394g7nnzmzo...,82
37,2026-02-09-01-03-26 +0000,nyt,Why the N.F.L. Booked Bad Bunny for the Super ...,https://www.nytimes.com/2026/02/08/business/ba...,80


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
14,101,2026-02-09-02-55-54 +0000,nypost,Trump slams Bad Bunny’s Super Bowl halftime sh...,https://nypost.com/2026/02/08/us-news/trump-sl...
59,63,2026-02-08-23-43-59 +0000,bbc,Japan's PM Takaichi on course to win snap elec...,https://www.bbc.com/news/articles/cx2y7d2z29xo...
17,37,2026-02-09-02-29-00 +0000,wsj,"Hong Kong’s Jimmy Lai Given 20-Year Sentence, ...",https://www.wsj.com/world/china/hong-kongs-jim...
130,36,2026-02-08-18-04-45 +0000,nypost,Iran sentences locked-up Nobel Peace Prize win...,https://nypost.com/2026/02/08/world-news/iran-...
134,33,2026-02-08-17-35-40 +0000,nypost,UK PM Keir Starmer’s top aide Morgan McSweeney...,https://nypost.com/2026/02/08/world-news/uk-pm...
171,32,2026-02-08-12-00-00 +0000,wsj,The Trump economy in 2025 was a roller coaster...,https://www.wsj.com/economy/consumers/american...
68,30,2026-02-08-23-01-00 +0000,startribune,Lindsey Vonn breaks her left leg in Olympic do...,https://www.startribune.com/2026-winter-olympi...
198,28,2026-02-08-09-00-00 +0000,cbc,"No desks, no strategy: Experts say government'...",https://www.cbc.ca/news/canada/ottawa/no-desks...
41,27,2026-02-09-00-55-57 +0000,nypost,California surfer found clinging to lobster po...,https://nypost.com/2026/02/08/us-news/californ...
181,25,2026-02-08-10-30-00 +0000,wsj,The pace of hiring in the U.S. has dropped off...,https://www.wsj.com/economy/jobs/this-is-why-i...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
